In [12]:
#import pandas as pd
#import string
#from matplotlib import rc

fontsize = 12

# Make default font serif
rc('font', family=b'serif', size=fontsize)
# Hide legend frame, change size, make only one scatter marker show up
rc('legend', borderpad=0.,labelspacing=0.25, fontsize=(fontsize - 1.), frameon=False, numpoints=1)
# Default line width (e.g. for plot()) should be thinner than 1 point
rc('patch', linewidth=0.75)
rc('lines', linewidth=0.75)

# BW1484 Core I Output

In [13]:
k_ref = 1.00020

### Load and parse the file

In [19]:
import re

# name of the file
out_file='bw1484_small.out' #$1
f = open('/home/slayer/Dropbox/Software/Exnihilo/Calculations/bw1484/'+out_file, 'r')

## Pairs of output variables and their keywords that always occur and only once
keys = ['problem_name', 'num cells I', 'num cells J', 'num cells K', 'cells', 'mesh ',
        'num_blocks_i', 'num_blocks_j', 'num_z_blocks', 'blocks', 'num_groups', 
        'num upscatter groups', 'num_sets', 'domains', 'Pn_order', 'azimuthals_octant', 
        'polars_octant', 'eq_set', 'DoF', 'tolerance', 'L2_tolerance', 'k_tolerance',
        'up_tol', 'eigen_solver', 'mg_preconditioner', 'relax_count', 'num_v_cycles', 
        'relax_weight', 'ml_Pn', 'ml_az', 'ml_oct']
vals = {key: None for key in keys}

# set flags and keys to use for items with duplicate entries
ml_flag = False
ml_keys = ['Pn_order', 'azimuthals_octant', 'polars_octant']
ml_doubles = {key: None for key in ml_keys}
ml_cnt = 0

up_db = False
doubles = {'tolerance' : None}

for line in f:
    
    # check for the duplicate items before processing the next stuff
    if ml_flag == True: 
        if ml_cnt != 3:
            for ml_key in ml_doubles:
                if re.search(ml_key, line):
                    if ml_doubles[ml_key] == None:
                        data = line.split(ml_key)
                        ml_doubles[ml_key] = data[1].strip() 
                        ml_cnt = ml_cnt + 1
        else:
            ml_flag = False
    if up_db == True:
        if re.search('tolerance', line):
            if doubles['tolerance'] == None:
                data = line.split('tolerance')
                doubles['tolerance'] = data[1].strip() 
                up_db = False
        
    for key in vals: 
        # check for things to set the flags
        if re.search('multilevel_quad_db', line):
            ml_flag = True
        if re.search('upscatter_db', line):
            up_db = True   
        # now look for the rest of the keys
        if re.search(key, line):
            data = line.split(key)
            vals[key] = data[1].strip()  
        
# calculate things we want in the spreadsheet
vals['cells'] = int(vals['num cells I']) * int(vals['num cells J']) * int(vals['num cells K'])
vals['blocks'] = int(vals['num_blocks_i']) * int(vals['num_blocks_j'])
vals['domains'] = int(vals['blocks']) * int(vals['num_sets'])
# set doubles
vals['ml_Pn'] = ml_doubles['Pn_order']
vals['ml_az'] = ml_doubles['azimuthals_octant']
vals['ml_oct'] = ml_doubles['polars_octant']
vals['up_tol'] = doubles['tolerance']

### print information to csv

In [20]:
import csv
with open('output.csv', 'wb') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = keys, dialect='excel', 
                            restval='',extrasaction='ignore')
    if not csv.Sniffer().has_header(csvfile.read(1024)):
        writer.writeheader()
    writer.writerow(vals)

IOError: File not open for reading